In [1002]:
import pandas as pd
import pandasql
import matplotlib.pyplot as plt
import os
from pandasql import sqldf
import seaborn as sns

In [1003]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [1004]:
def load_query(path):
    with open(path) as file:
        query=file.read()
    return query    
pysql=lambda q: sqldf(q,globals())

In [1005]:
consumption_data=pd.read_csv('Yearly_consumption_change.csv')
tax_data=pd.read_csv('Yearly_tax_increase_percentage.csv')

In [1013]:
consumption_data[consumption_data['State']=='Louisiana'].head(10)

,State,Year,Gender,Race,Age,Education,Current_Consumption,Previous_Year_Consumption,Current_Everyday_Consumption,Previous_Everyday_Consumption
19,Louisiana,2012,Overall,All Races,All Ages,All Grades,24.8,25.7,18.6,19.3
72,Louisiana,2013,Overall,All Races,All Ages,All Grades,23.5,24.8,16.7,18.6
125,Louisiana,2014,Overall,All Races,All Ages,All Grades,24.0,23.5,16.8,16.7
178,Louisiana,2015,Overall,All Races,All Ages,All Grades,21.9,24.0,15.2,16.8
231,Louisiana,2016,Overall,All Races,All Ages,All Grades,22.8,21.9,15.2,15.2
284,Louisiana,2017,Overall,All Races,All Ages,All Grades,23.1,22.8,15.1,15.2
337,Louisiana,2018,Overall,All Races,All Ages,All Grades,20.5,23.1,13.6,15.1
390,Louisiana,2019,Overall,All Races,All Ages,All Grades,21.9,20.5,15.6,13.6


In [1014]:
tax_data[tax_data['State']=='Louisiana']

,State,Year,Current_Tax,Previous_Year_Tax,Percent_Tax_Increase
18,Louisiana,2012,1.37,1.37,0.000000
69,Louisiana,2013,1.37,1.37,0.000000
120,Louisiana,2014,1.37,1.37,0.000000
171,Louisiana,2015,1.87,1.37,36.496350
222,Louisiana,2016,2.09,1.87,11.764706
273,Louisiana,2017,2.09,2.09,0.000000
324,Louisiana,2018,2.09,2.09,0.000000
375,Louisiana,2019,2.09,2.09,0.000000


In [1007]:
elasticity_query=load_query('Elasticity.sql')
elasticity_study_data=pysql(elasticity_query)

In [1008]:
elasticity_data=elasticity_study_data[(elasticity_study_data['Race']=='All Races') &(elasticity_study_data['Gender']=='Overall')&(elasticity_study_data['Age']=='All Ages')]
#elasticity_data=elasticity_study_data[elasticity_study_data['Gender']=='Overall']

In [1009]:
elasticity_study_data.to_csv('Elasticity_Study_Data.csv',index=False)

### DIFFERENCE IN DIFFERENCE ESTIMATOR

In [812]:
DID_control_cases_query=load_query('DIDNEW.sql')
DID_control_cases=pysql(DID_control_cases_query)
DID_control_cases['Treat'] = 0
DID_control_cases['Post'] = (DID_control_cases['Year'] >2015).astype(int)

In [813]:
DID_treat_cases_query=load_query('DIDNEW2.sql')
DID_treat_cases=pysql(DID_treat_cases_query)
DID_treat_cases['Treat'] = 1
DID_treat_cases['Post'] = (DID_treat_cases['Year'] >2015).astype(int)

In [814]:
Data=pd.concat([DID_treat_cases,DID_control_cases],axis=0)
Data.loc[Data['Year'] == 2015, 'Tax_Change'] = 0
Data.reset_index(drop=True).head(10)
Data.dropna(inplace=True)
Data.sort_values(by=['Treat'],ascending=False).head(5)

,Year,State,Current_Consumption,Previous_Year_Consumption,Current_Tax,Previous_Year_Tax,Tax_Change,Treat,Post
0,2012,Alabama,23.8,24.3,1.435,1.435,0.00,1,0
34,2017,Vermont,15.8,17.0,4.090,4.090,0.46,1,1
25,2016,Kansas,17.2,17.7,2.300,2.300,0.50,1,1
26,2016,Nevada,16.5,17.5,2.810,2.810,1.00,1,1
27,2016,Ohio,22.5,21.6,2.610,2.610,0.35,1,1


### Selecting States with similar consumptions durin 2015 for DID Study

In [815]:
Data=Data[(Data['State']=='Nebraska')|(Data['State']=='Wisconsin')|(Data['State']=='Nevada')|(Data['State']=='Kansas')]
Data=Data[(Data['Year']==2015)|(Data['Year']==2016)]
Data

,Year,State,Current_Consumption,Previous_Year_Consumption,Current_Tax,Previous_Year_Tax,Tax_Change,Treat,Post
19,2015,Kansas,17.7,18.1,2.30,1.80,0.0,1,0
20,2015,Nevada,17.5,17.0,2.81,1.81,0.0,1,0
25,2016,Kansas,17.2,17.7,2.30,2.30,0.5,1,1
26,2016,Nevada,16.5,17.5,2.81,2.81,1.0,1,1
140,2015,Nebraska,17.1,17.3,1.65,1.65,0.0,0,0
158,2015,Wisconsin,17.3,17.4,3.53,3.53,0.0,0,0
181,2016,Nebraska,17.0,17.1,1.65,1.65,0.0,0,1
196,2016,Wisconsin,17.1,17.3,3.53,3.53,0.0,0,1


In [816]:
import pandas as pd
import statsmodels.api as sm

# Assuming DiD is your DataFrame
Data['Treatment_Post'] = Data['Treat'] * Data['Post']
state_dummies = pd.get_dummies(Data['State'], dtype=float,drop_first=True)
Data.dropna(inplace=True)
# Define the independent variables (X) and dependent variable (y)
#X = sm.add_constant(pd.concat([Data[['Post','Tax_Change']], state_dummies], axis=1))
X = sm.add_constant(Data[['Treat','Post','Tax_Change']])
y = Data['Current_Consumption']

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Display the regression summary
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     Current_Consumption   R-squared:                       0.915
Model:                             OLS   Adj. R-squared:                  0.851
Method:                  Least Squares   F-statistic:                     14.34
Date:                 Tue, 19 Dec 2023   Prob (F-statistic):             0.0132
Time:                         19:55:29   Log-Likelihood:                 7.2663
No. Observations:                    8   AIC:                            -6.533
Df Residuals:                        4   BIC:                            -6.215
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.1654      0.094    183.11